# Collect bookspines
Collect from firestore and storage

In [1]:
#firestore

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import os, json

cred = credentials.Certificate(r"avea-biblosphere-210106-firebase-adminsdk-s3bjc-ff2c6ac275 (1).json")
firebase_admin.initialize_app(cred)

db = firestore.client()

In [2]:
# storage
from google.cloud import storage

storage_client = storage.Client.from_service_account_json('biblosphere-210106-0e61a72c6976.json')
bucket = storage_client.get_bucket('biblosphere-210106.appspot.com')

In [3]:
#bigquery
from google.oauth2 import service_account
from google.cloud import bigquery

project_id = 'biblosphere-210106'
credentials = service_account.Credentials.from_service_account_file('biblosphere-210106-0e61a72c6976.json')

bq_client = bigquery.Client(credentials= credentials,project=project_id)

In [4]:
#create table
from google.cloud import bigquery

#client = bigquery.Client()

table_id = "biblosphere-210106.biblosphere.finded_unfinded_books"

schema = [
    bigquery.SchemaField("photo_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("result", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("bookspine", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("url", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("authors", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("image", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("isbn", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("keys", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("language", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("title", "STRING", mode="NULLABLE"),
]

table = bigquery.Table(table_id, schema=schema)
table = bq_client.create_table(table)
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table biblosphere-210106.biblosphere.finded_unfinded_books


In [5]:
def get_books_from_bucket(data):
    filename = data['photo']
    json_filename = os.path.splitext(filename)[0] + '.json'
    jsonfile_blob = bucket.blob(json_filename)

    json_data = json.loads(jsonfile_blob.download_as_string())
    recognized = json_data['recognized'] if 'recognized' else []
    unrecognized = json_data['unrecognized'] if 'unrecognized' else []

    return recognized, unrecognized

def insert_to_bigquery(record):
    errors = bq_client.insert_rows_json(table_id, record)  
    if errors != []:
        print(errors)

In [ ]:
res = db.collection('photos').where(u'recognition_stats.record_in_stats', u'==', True).stream()

count = 0
for n, doc in enumerate(res):
#     count += 1
#     if count > 1:
#         break

    photo_id = doc.id
    print(f'{n}) {photo_id}')
    data = doc.to_dict()
    url = data['url']
    recognized, unrecognized = get_books_from_bucket(data)


    records = []
    for book in recognized:
        record = {u'photo_id': photo_id,
                u'result': 'recognized',
                u'bookspine': book['bookspine'],
                u'url': url,
                u'authors': book['book']['authors'],
                u'image': book['book']['image'],
                u'isbn': book['book']['isbn'],
                u'keys': ' '.join(book['book']['keys']),
                u'language': book['book']['language'],
                u'title': book['book']['title'],
                }
        records.append(record)
      
    for book in unrecognized:
        record = {u'photo_id': photo_id,
                 u'result': 'unrecognized',
                 u'bookspine': book['bookspine'],
                 u'url': url
                 }
        records.append(record)
    
    try:
        insert_to_bigquery(records)
    except:
        print('error:', photo_id)
        
        